# Testing C++ code in python
June 12, 2020

In [15]:
import numpy as np

Steps to implement spectrum calculation
- Compute FFT
- Shift frequencies
- Compute radial average

## Save image for C++ code

In [87]:
xsize,ysize=128,128

In [96]:
### Save single 2D image to csv for C++ code to handle
fname='data/gen_images.npy'
a1=np.load(fname)
print(a1.shape)
np.savetxt('data/images.csv',a1[10,0,:xsize,:ysize],delimiter=',',newline=',')

(512, 1, 128, 128)


In [97]:
def f_radial_profile(data, center=(None,None)):
    ''' Module to compute radial profile of a 2D image '''
    y, x = np.indices((data.shape)) # Get a grid of x and y values
    
    if center[0]==None and center[1]==None:
        center = np.array([(x.max()-x.min())/2.0, (y.max()-y.min())/2.0]) # compute centers
        
    # get radial values of every pair of points
    r = np.sqrt((x - center[0])**2 + (y - center[1])**2)
    r = r.astype(np.int)
    
    # Compute histogram of r values
    tbin = np.bincount(r.ravel(), data.ravel())
    nr = np.bincount(r.ravel()) 
    radialprofile = tbin / nr
    
    return radialprofile

## Read input

In [98]:
fname='data/images.csv'
x=np.loadtxt(fname,delimiter=',',dtype=str)[:-1].astype(np.float64).reshape(xsize,ysize)
print(x.shape)
# x

(128, 128)


In [99]:
# x=np.array([[23,6,17,35,33],[15,26,12,9,21],[2,27,10,19,3],[6,20,26,12,16],[11,8,7,29,22]])
# x

In [100]:
y1=np.fft.fft2(x)
y2=abs(y1)
# y2

# for i in y2: 
#     for j in i:
#         print(np.around(j,decimals=6),end='\t')
#     print('')
    
    
z1=f_radial_profile(y2)
print(z1)

[  10.49694482    3.40802768    4.76252278    6.36349742    5.10413934
    4.79756468    4.97840299    5.39588531    5.4003053     4.53464022
    5.49957069    5.53165724    5.71546969    5.27360164    5.18619513
    6.30545474    6.09051821    6.0685409     6.40990659    6.04353769
    5.74806914    5.81036689    6.12536387    6.8247455     6.94928425
    6.92604092    6.84612515    7.08641838    7.65645585    7.05711512
    6.60270502    7.22703528    8.41921883    8.68995137    8.32957115
    9.26714528    9.20884492    8.67788399    8.92253353    9.84759988
   10.14208274   10.37489039   10.2559299    10.69001194   11.94675542
   11.52709725   11.42025112   11.89112413   12.08412585   12.06162137
   13.01154185   13.77082109   13.74344759   14.36494818   14.17852331
   15.87449192   17.45427333   17.1312081    17.04753819   17.7287977
   19.09573394   19.1902872    18.82942227   20.3700787    23.1163633
   27.82357658   29.78915542   30.87633173   33.16629766   32.9974442
   37.664

In [101]:
### Read c++ file
cp_ip_file='data/op.csv'
# x=np.loadtxt(fname,delimiter=',',dtype=str)[:-1].astype(np.float64).reshape(5,5)
z_cpp=np.loadtxt(cp_ip_file,delimiter=',',dtype=str)[:-1].astype(np.float64)
print(z_cpp)

[  10.4969     3.40803    4.76252    6.3635     5.10414    4.79756
    4.9784     5.39589    5.40031    4.53464    5.49957    5.53166
    5.71547    5.2736     5.1862     6.30545    6.09052    6.06854
    6.40991    6.04354    5.74807    5.81037    6.12536    6.82475
    6.94928    6.92604    6.84613    7.08642    7.65646    7.05712
    6.60271    7.22704    8.41922    8.68995    8.32957    9.26715
    9.20884    8.67788    8.92253    9.8476    10.1421    10.3749
   10.2559    10.69      11.9468    11.5271    11.4203    11.8911
   12.0841    12.0616    13.0115    13.7708    13.7434    14.3649
   14.1785    15.8745    17.4543    17.1312    17.0475    17.7288
   19.0957    19.1903    18.8294    20.3701    23.1164    27.8236
   29.7892    30.8763    33.1663    32.9974    37.6647    39.8517
   42.6102    46.6871    49.6994    53.2687    59.9486    65.6641
   64.5987    75.9153    84.4468    74.5143   101.476    102.473
  130.016    156.813    151.878    214.875    293.714   1287.41
    0. 

In [102]:
### Check arrays
np.allclose(z1,z_cpp[:z1.shape[0]],rtol=1e-4,atol=1e-8)

True

In [103]:
### Check manually elementwise
for i in range(z1.shape[0]):
    if (np.around(z1[i],4)!=np.around(z_cpp[i],4)): 
          print(i,z1[i],z_cpp[i])

23 6.82474549864186 6.82475
35 9.267145277973935 9.26715
82 101.47550457664987 101.476
83 102.47313527901589 102.473
84 130.01640406575825 130.016
85 156.81321422541942 156.813
86 151.87788543886876 151.878
87 214.87473325471487 214.875
88 293.71409386086765 293.714
89 1287.4118356440815 1287.41


## Conclusion:
###### July 2, 2020
Reading input + computing FFT + computing Modulus + Getting Radial profile \
Arrays from python and C++ match !!\
The C++ code is correct and ready to port to LBANN